In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [31]:
import re
import json
from datasets import Dataset

def noktalama_kaldir(metin):
    # Yaygın noktalama işaretlerini kaldır
    metin = re.sub(r'[^\w\s]', '', metin)
    return metin

# Verisetinizin bulunduğu yolu belirtin
veri_yolu = '/content/fikralarFinal.json' # Verisetinizin dosya adı ve uzantısını doğru belirtin

with open(veri_yolu, 'r', encoding='utf-8') as f:
    veri = json.load(f)

temizlenmis_fikralar = []
for kayit in veri:
    if "icerik" in kayit:
        temizlenmis_fikralar.append(noktalama_kaldir(kayit["icerik"]))

temizlenmis_dataset = Dataset.from_dict({"text": temizlenmis_fikralar})

print(f"Toplam temizlenmiş fıkra sayısı: {len(temizlenmis_fikralar)}")
print("İlk birkaç temizlenmiş fıkra:", temizlenmis_dataset[:5])

# Sonraki adımlarda bu 'temizlenmis_dataset'i kullanabilirsiniz.

Toplam temizlenmiş fıkra sayısı: 1207
İlk birkaç temizlenmiş fıkra: {'text': ['Fıkra Metni', 'Nasreddin Hoca ya dert yaniyorlar Yahu Hoca senin kari çok geziyor Hoca  Olur mu canim O kadar gezse arada bir bizim eve de ugrar', 'Iki deli havuzun basinda oturuyorlarmisBiri kalkip havuza seker atmisHavuzdan bir yudum almis ve tükürmüsArkadasina Havuza seker attim ama tatli olmadi Arkadasi Karistirmadinki salak', 'Iki deli birgün deliler hastanesinden kaçmislarKimse bu delileri bulamamislarDoktorlar ümitlerini kestikleri an deliler çika gelmisDoktorlar hayretle niye geldiniz demis DelilerYarin kaçacagizda onun provasini yaptik', 'Delinin biri yolun kenarindaki uçurumda durmus asagiya bakarak 13 13 13 diye soyleniyormus Oradan gecen biri delinin ne yaptigini merak etmis yanasarak  ne yapi diyemeden deli onu birden uçurumdan asagiya ativermis ve devam etmis 14 14 14']}


In [32]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_adi = "ytu-ce-cosmos/turkish-gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_adi)
model = GPT2LMHeadModel.from_pretrained(model_adi)

# Özel bir <|endoftext|> tokeni ekleyebiliriz (isteğe bağlı)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [33]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1207 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [34]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False # mlm=False (Masked Language Modeling) yerine sadece Language Modeling yapıyoruz
)

In [40]:
from transformers import TrainingArguments

output_model_adi = "gpt2-fikra-uretici"
output_dir = f"./{output_model_adi}"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=6,
    per_device_train_batch_size=4,
    save_steps=100,
    save_total_limit=2,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    lr_scheduler_type= 'cosine',
    push_to_hub=False,
)


In [41]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

Step,Training Loss
500,0.641400
1000,0.555400
1500,0.538300


('./gpt2-fikra-uretici/tokenizer_config.json',
 './gpt2-fikra-uretici/special_tokens_map.json',
 './gpt2-fikra-uretici/vocab.json',
 './gpt2-fikra-uretici/merges.txt',
 './gpt2-fikra-uretici/added_tokens.json')

In [43]:
def fikra_uret(prompt, max_length=50, num_return_sequences=1, temperature=0.5, prefix=""):
    cikti = fikra_uretici(prefix + prompt,
                        max_length=max_length,
                        num_return_sequences=num_return_sequences,
                        temperature=temperature,
                        do_sample=True,
                        top_k=50,
                        top_p=0.95)
    for i, sonuc in enumerate(cikti):
        print(f"Üretilen Fıkra {i+1}: {sonuc['generated_text']}")

# Örnek fıkra üretimi
baslangic = "Keloğlan"
fikra_uret(baslangic)

#baslangic2 = "Temel ile Dursun bir gün:"
fikra_uret(baslangic2, max_length=50, temperature=0.6)

Üretilen Fıkra 1: Keloğlan bir gün bir dükkana gitmiş:-Bana bir ekmek verin.-Dükkan sahibi:-Kaç para var?-Keloğlan:-Hiç yok ama ekmeğin kokusunu alırım!; demiş;-Öyleyse neden ekmeğin kokusunu almayasın?;;-Çünkü
Üretilen Fıkra 1: Temel ile Dursun bir gün: - Uşağum oruçlu oruçlu kaç hamsi yiyepilursun? - 100 tane demiş. - Hadi oradan yesen yesen 1 tane yersin geriye kalan 99 hamsiyi oruçsuz yersin demiş


In [45]:
import os
print(os.listdir('./'))

['.config', 'wandb', 'drive', 'fikralarFinal.json', 'gpt2-fikra-uretici', 'sample_data']


In [ ]:
!zip -r gpt2-fikra-model.zip ./gpt2-fikra-uretici



  adding: gpt2-fikra-uretici/ (stored 0%)
  adding: gpt2-fikra-uretici/config.json (deflated 52%)
  adding: gpt2-fikra-uretici/generation_config.json (deflated 31%)
  adding: gpt2-fikra-uretici/runs/ (stored 0%)
  adding: gpt2-fikra-uretici/runs/May11_18-00-26_0282afadad7d/ (stored 0%)
  adding: gpt2-fikra-uretici/runs/May11_18-00-26_0282afadad7d/events.out.tfevents.1746986429.0282afadad7d.397.2 (deflated 62%)
  adding: gpt2-fikra-uretici/runs/May11_17-48-41_0282afadad7d/ (stored 0%)
  adding: gpt2-fikra-uretici/runs/May11_17-48-41_0282afadad7d/events.out.tfevents.1746985739.0282afadad7d.397.0 (deflated 60%)
  adding: gpt2-fikra-uretici/runs/May11_17-48-41_0282afadad7d/events.out.tfevents.1746986252.0282afadad7d.397.1 (deflated 62%)
  adding: gpt2-fikra-uretici/runs/May11_18-51-21_0282afadad7d/ (stored 0%)
  adding: gpt2-fikra-uretici/runs/May11_18-51-21_0282afadad7d/events.out.tfevents.1746989483.0282afadad7d.397.8 (deflated 60%)
  adding: gpt2-fikra-uretici/runs/May11_18-38-11_0282af

In [ ]:
from google.colab import files
files.download("gpt2-fikra-model.zip")
